# Naive Bayes (Bayes Ingenuo) - Con tunneo de hiperparámetros


In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import pandas as pd

#### Carga de datos y preparación de variables


In [2]:
df = pd.read_csv("train.csv")

variables_numericas = [
'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
    '1stFlrSF', 'FullBath', 'YearBuilt', 'KitchenAbvGr',
    'TotRmsAbvGrd', 'Fireplaces', 'SalePrice'
]

df = df[variables_numericas]

#### Definir X (variables predictoras) y "y" (variable respuesta)


In [3]:
p40 = df['SalePrice'].quantile(0.40)
p90 = df['SalePrice'].quantile(0.90)

def clasificar_precio(precio: float) -> str:
    if precio <= p40:
        return "Económica"
    elif precio <= p90:
        return "Intermedia"
    else:
        return "Cara"

df['CategoriaPrecio'] = df['SalePrice'].apply(clasificar_precio)

X = df.drop(['SalePrice', 'CategoriaPrecio'], axis=1)
y = df['CategoriaPrecio']

#### Dividir datos en entrenamienta y prueba


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Definir hiperparámetros para el modelo


In [7]:
param_grid_naive_bayes =  [{
    "var_smoothing": np.logspace(0,-9, num=100)
}]


bayes_classifier = GridSearchCV(GaussianNB(), param_grid=param_grid_naive_bayes, n_jobs=2, cv=5, scoring='accuracy')
bayes_classifier.fit(X_train, y_train)
best_bayes_classifier = bayes_classifier.best_estimator_

model = best_bayes_classifier

#### Evaluación de Modelo en Conjunto de Prueba


In [8]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')

print(f" - Bayes Regressor (Tunneo de Hiperparámetros):")
print(f"   - Accuracy: {accuracy:.2f}")
print(f"   - Precision: {precision:.2f}")
print(f"   - Recall: {recall:.2f}")
print(f"   - F1-score: {f1:.2f}")
print("-" * 40)

 - Bayes Regressor (Tunneo de Hiperparámetros):
   - Accuracy: 0.84
   - Precision: 0.84
   - Recall: 0.84
   - F1-score: 0.84
----------------------------------------


<small>

### **Conclusión: Modelo de Clasificación (Naïve Bayes)**

El ajuste de hiperparámetros en el **modelo de clasificación** también resultó en **una mejora en todas las métricas de evaluación**:

- **Precisión (Accuracy)** aumentó de **0.82 a 0.84**, lo que significa que el modelo clasifica correctamente un mayor porcentaje de viviendas.
- **Recall, Precisión y F1-score** mejoraron de **0.82 a 0.84**, lo que sugiere que el modelo es más **equilibrado** en términos de su capacidad para identificar correctamente cada categoría de precios y evitar falsos positivos o negativos.

**Este incremento en el rendimiento sugiere que el modelo ajustado tiene mejor capacidad para diferenciar correctamente entre casas económicas, intermedias y caras.** La optimización ayudó a reducir los errores de clasificación y a hacer el modelo más estable en sus predicciones.

### **Conclusión Global**

El proceso de **ajuste de hiperparámetros** logró **mejorar ambos modelos**.

- En **regresión**, el modelo ahora comete **errores menos graves y predice valores más cercanos a los reales**.
- En **clasificación**, el modelo ahora **identifica mejor las clases y reduce la cantidad de errores de categorización**.

</small>
